# IMDbMovies

## Contents <a id='back'></a>

1. [Introduction](#introduction)
2. [Loading the Dataset](#loading-the-dataset)
3. [Data Cleaning](#data-cleaning)
   - 3.1 [Header style](#header-style)
   - 3.2 [Handling Missing Values](#handling-missing-values)
   - 3.3 [Data Transformation](#data-transformation)
4. [Exploratory Data Analysis (EDA)](#exploratory-data-analysis-eda)
   - 4.1 [Overview of the Dataset](#overview-of-the-dataset)
   - 4.2 [Visualizing Data](#visualizing-data)

## 1. Introduction <a name="introduction"></a>

The IMDbMovies dataset is a comprehensive collection with information on more than 9000 movies on IMDb. The dataset includes key features such as title, director, writer, genres, runtime, release year, budget, and gross revenue. 

The primary goal of this analysis is to gain a deeper insight of the evolution of the cinematographic industry over the years.

[Back to contents](#back)

## 2. Loading the Dataset <a name="loading-the-dataset"></a>

The IMDB Movies dataset is available on [kaggle.com](https://www.kaggle.com/datasets/elvinrustam/imdb-movies-dataset). We will be using the unclean version, so we can carry out our own cleanup. 

The first step is to correct a few rows that have the correct data, but it's shifted one column to the right. The movies that require this are:
 
- Sleepy Hollow
- Texas Chainsaw
- G.I. Joe: Retaliation
- The Woman in the Window
- The Social Dilemma
- Bank of Dave
- Eden
- CHIPS
- The Diving Bell and the Butterfly
- Untamed Heart
- Roxanne
- The Postcard Killings
- Cover Girl

We'll be fixing this with google docs.

Note that "The Fall of Minneapolis" has a similar issue, but also a lot of missing values, and the value for budget (302) doesn't make sense. We'll drop this row. 

The rest of the preprocessing will be done here. We can load the Dataset now.

In [1]:
import pandas as pd

df = pd.read_csv("../IMDbMovies.csv")

Lets see what the data looks like.

In [2]:
df.head(5)

,Title,Summary,Director,Writer,Main Genres,Motion Picture Rating,Runtime,Release Year,Rating,Number of Ratings,Budget,Gross in US & Canada,Gross worldwide,Opening Weekend Gross in US & Canada
0,Napoleon,An epic that details the checkered rise and fa...,Ridley Scott,David Scarpa,"Action,Adventure,Biography",R,2h 38m,2023.0,6.7/10,38K,NaN,"$37,514,498","$84,968,381","$20,638,887Nov 26, 2023"
1,The Hunger Games: The Ballad of Songbirds & Sn...,Coriolanus Snow mentors and develops feelings ...,Francis Lawrence,"Michael Lesslie,Michael Arndt,Suzanne Collins","Action,Adventure,Drama",PG-13,2h 37m,2023.0,7.2/10,37K,"$100,000,000 (estimated)","$105,043,414","$191,729,235","$44,607,143Nov 19, 2023"
2,The Killer,"After a fateful near-miss, an assassin battles...",David Fincher,"Andrew Kevin Walker,Luc Jacamon,Alexis Nolent","Action,Adventure,Crime",R,1h 58m,2023.0,6.8/10,117K,NaN,NaN,"$421,332",NaN
3,Leo,A 74-year-old lizard named Leo and his turtle ...,"David Wachtenheim,Robert Smigel,Robert Marianetti","Paul Sado,Robert Smigel,Adam Sandler","Animation,Comedy,Family",PG,1h 42m,2023.0,7.0/10,10K,NaN,NaN,NaN,NaN
4,Thanksgiving,"After a Black Friday riot ends in tragedy, a m...",Eli Roth,"Eli Roth,Jeff Rendell","Horror,Mystery,Thriller",R,1h 46m,2023.0,7.0/10,9.1K,NaN,"$25,408,677","$29,666,585","$10,306,272Nov 19, 2023"


There seems to be a lot of missing values on the `Budget` and `Gross` columns, and some columns are in a format that is uncomfortable to work with. Time for cleanup.

[Back to contents](#back)

## 3. Data Cleaning <a name="data-cleaning"></a>

### 3.1 Header style <a name="header-style"></a>

First we'll fix the names of the existing columns.

In [3]:
base_columns_names = df.columns
base_columns_names

Index(['Title', 'Summary', 'Director', 'Writer', 'Main Genres',
       'Motion Picture Rating', 'Runtime', 'Release Year', 'Rating',
       'Number of Ratings', 'Budget', 'Gross in US & Canada',
       'Gross worldwide', 'Opening Weekend Gross in US & Canada'],
      dtype='object')

In [4]:
fixed_columns_names = {}

for name in base_columns_names:
    fixed_name = name.lower()
    fixed_name = fixed_name.replace(' & ', '&') #turns "us & canada" into "us&canada", to avoid so many '_' characters in the next step
    fixed_name = fixed_name.replace(' ', '_')
    fixed_columns_names [name] = fixed_name

fixed_columns_names

{'Title': 'title',
 'Summary': 'summary',
 'Director': 'director',
 'Writer': 'writer',
 'Main Genres': 'main_genres',
 'Motion Picture Rating': 'motion_picture_rating',
 'Runtime': 'runtime',
 'Release Year': 'release_year',
 'Rating': 'rating',
 'Number of Ratings': 'number_of_ratings',
 'Budget': 'budget',
 'Gross in US & Canada': 'gross_in_us&canada',
 'Gross worldwide': 'gross_worldwide',
 'Opening Weekend Gross in US & Canada': 'opening_weekend_gross_in_us&canada'}

In [5]:
df = df.rename(columns = fixed_columns_names)
df.columns

Index(['title', 'summary', 'director', 'writer', 'main_genres',
       'motion_picture_rating', 'runtime', 'release_year', 'rating',
       'number_of_ratings', 'budget', 'gross_in_us&canada', 'gross_worldwide',
       'opening_weekend_gross_in_us&canada'],
      dtype='object')

With the columns renamed, lets move on to improvind the format of certain columns. We might end up renaming some of them again.

[Back to contents](#back)

### 3.2 Handling Missing Values <a name="handling-missing-values"></a>

First, lets see how many missing values we have to deal with.

In [6]:
df.isna().sum()

title                                    0
summary                                  0
director                                31
writer                                 324
main_genres                              7
motion_picture_rating                  796
runtime                                165
release_year                             8
rating                                 269
number_of_ratings                      270
budget                                3203
gross_in_us&canada                    3018
gross_worldwide                       1954
opening_weekend_gross_in_us&canada    3387
dtype: int64

Many of these are not critical. We can fill them with `'unknown'` for text columns, or `-1` in numerical columns, to mark the data as unavailable, since none of these columns should naturally have negative values.

The columns `director`, `writer`, `main_genres`, and `motion_picture_rating` can be filled with the value `'unknown'`

In [7]:
columns_to_fix = ['director', 'writer', 'main_genres', 'motion_picture_rating']

for column in columns_to_fix:
    df[column] = df[column].fillna(value = 'unknown')

The missing values in the columns `rating`, `number_of_ratings`, `budget`, `gross_in_us&canada`, `gross_worldwide`, and `opening_weekend_gross_in_us&canada` can be filled, but we need to be careful to keep their general format, so the new values don't interfere with data processing.

In [9]:
# rating is in the format (value/10)
df['rating'] = df['rating'].fillna(value = '0/10')
df['number_of_ratings'] = df['number_of_ratings'].fillna(value = 0)

In [10]:
# runtime is expressed in hours (h) and minutes (m)
df['runtime'] = df['runtime'].fillna(value = '0m')

In [11]:
# all entries have a currency, and $ is the most common
columns_to_fix = ['budget', 'gross_in_us&canada', 'gross_worldwide']

for column in columns_to_fix:
    df[column] = df[column].fillna(value = '$0') 

In [12]:
# all entries have the $ at the beginning, followed by the date of the opening weekend
df['opening_weekend_gross_in_us&canada'] = df['opening_weekend_gross_in_us&canada'].fillna(value = '$0Jan 1, 1900')

The `release_year` is going to be critical. We drop rows that are missing this value.

In [13]:
df.dropna(subset = ['release_year'], inplace=True)

In [14]:
df.isna().sum()

title                                 0
summary                               0
director                              0
writer                                0
main_genres                           0
motion_picture_rating                 0
runtime                               0
release_year                          0
rating                                0
number_of_ratings                     0
budget                                0
gross_in_us&canada                    0
gross_worldwide                       0
opening_weekend_gross_in_us&canada    0
dtype: int64

No more missing values. Let's check duplicates.

In [15]:
df.duplicated().sum()

0

No obvious duplicates.

Just to make sure, lets double check. We could find duplicates in `title`, but it could still be from different movies. A duplicated `summary` would be more suspicious.

In [29]:
df['summary'].duplicated().sum()

32

There are some duplicates here. Lets see what they are.

In [34]:
dup = df[df['summary'].duplicated(keep=False) == True]
dup = dup.sort_values('summary')

dup


,title,summary,director,writer,main_genres,motion_picture_rating,runtime,release_year,rating,number_of_ratings,budget,gross_in_us&canada,gross_worldwide,opening_weekend_gross_in_us&canada
7803,Aladdin,A kind-hearted street urchin and a power-hungr...,Guy Ritchie,"John August,Guy Ritchie","Adventure,Comedy,Family",PG,2h 8m,2019.0,6.9/10,284K,"$183,000,000 (estimated)","$355,559,216","$1,054,304,000","$91,500,929May 26, 2019"
557,Aladdin,A kind-hearted street urchin and a power-hungr...,"John Musker,Ron Clements","Ron Clements,John Musker,Ted Elliott","Animation,Adventure,Comedy",G,1h 30m,1992.0,8.0/10,454K,"$28,000,000 (estimated)","$217,350,219","$504,050,219","$196,664Nov 15, 1992"
5521,We're the Millers 2,Plot is unknown.,Rawson Marshall Thurber,Adam Sztykiel,Comedy,R,2h 3m,2001.0,0/10,0,$0,$0,$0,"$0Jan 1, 1900"
6403,Real Steel 2,Plot is unknown.,unknown,John Gatins,"Action,Drama,Family",Unrated,1h 43m,2022.0,0/10,0,$0,$0,$0,"$0Jan 1, 1900"
3459,Godzilla x Kong: The New Empire,Plot kept under wraps.,Adam Wingard,"Simon Barrett,Jeremy Slater,Terry Rossio","Action,Adventure,Sci-Fi",unknown,0m,2024.0,0/10,0,$0,$0,$0,"$0Jan 1, 1900"
5536,Iron Lung,Plot kept under wraps.,Mark Fischbach,"Mark Fischbach,David Szymanski","Horror,Sci-Fi",unknown,2h 1m,2022.0,0/10,0,$0,$0,$0,"$0Jan 1, 1900"
2302,Fast X: Part 2,Plot kept under wraps.,Louis Leterrier,"Oren Uziel,Christina Hodson","Action,Crime,Thriller",unknown,0m,2025.0,0/10,0,$0,$0,$0,"$0Jan 1, 1900"
1144,Lethal Weapon 5,Plot kept under wraps.,Mel Gibson,"Shane Black,Richard Wenk","Action,Thriller",R,1h 55m,2018.0,0/10,0,$0,$0,$0,"$0Jan 1, 1900"
5319,Kind of Kindness,Plot kept under wraps.,Yorgos Lanthimos,"Efthimis Filippou,Yorgos Lanthimos",unknown,unknown,0m,2024.0,0/10,0,$0,$0,$0,"$0Jan 1, 1900"
8322,Untitled Margot Robbie Ocean's Eleven Film,Plot under wraps,Jay Roach,"Jack Golden Russell,George Clayton Johnson,Car...",unknown,Not Rated,2h 5m,1955.0,0/10,0,$0,$0,$0,"$0Jan 1, 1900"


Most of the duplicates are from placeholder text. There are two actual duplicated summaries for the movies 'Aladdin' and 'Lady and the Tramp', but correspond to the original film and their remake.

We can be reasonably sure that there are no duplicates in the DataSet.

[Back to Content](#back)

### 3.3 Data Transformation <a name="data-transformation"></a>



The runtime is presented in hours + minutes. This format is useful for display, but for comparisons and calculations the runtime in minutes would be much better.

In [16]:
runtime_minutes=[]

for time in df['runtime']:
    hours = 0
    h_pos = time.find('h')
    print (time)
    if h_pos > 0:
        hours = int(time[:h_pos])
    print(hours)
    #minutes = int(hours_min[])

2h 38m
2
2h 37m
2
1h 58m
1
1h 42m
1
1h 46m
1
3h
3
2h 11m
2
1h 45m
1
1h 35m
1
2h 13m
2
3h 26m
3
1h 31m
1
1h 49m
1
2h 7m
2
2h 22m
2
0m
0
1h 43m
1
2h 34m
2
2h 13m
2
1h 54m
1
1h 40m
1
1h 32m
1
1h 22m
1
1h 52m
1
1h 49m
1
1h 43m
1
1h 55m
1
1h 52m
1
2h 44m
2
1h 37m
1
1h 33m
1
2h 43m
2
2h 20m
2
1h 30m
1
1h 37m
1
1h 33m
1
1h 43m
1
1h 45m
1
2h 26m
2
2h 21m
2
2h 15m
2
1h 42m
1
1h 53m
1
1h 44m
1
2h 1m
2
1h 57m
1
2h 3m
2
2h 5m
2
2h 9m
2
1h 44m
1
1h 49m
1
2h 49m
2
1h 33m
1
2h 14m
2
2h 13m
2
2h 31m
2
2h 11m
2
1h 35m
1
2h 32m
2
0m
0
1h 32m
1
2h 55m
2
1h 55m
1
2h 49m
2
1h 46m
1
2h 49m
2
1h 44m
1
1h 45m
1
1h 35m
1
2h 17m
2
0m
0
0m
0
1h 25m
1
1h 40m
1
1h 39m
1
2h 3m
2
2h 22m
2
1h 58m
1
0m
0
1h 23m
1
2h 35m
2
1h 34m
1
2h 35m
2
2h 10m
2
2h 2m
2
2h 56m
2
2h 14m
2
1h 41m
1
2h 30m
2
2h 34m
2
1h 42m
1
2h 14m
2
2h 4m
2
2h 4m
2
1h 32m
1
1h 43m
1
3h
3
2h 4m
2
1h 36m
1
1h 41m
1
1h 53m
1
2h 7m
2
1h 57m
1
1h 40m
1
1h 47m
1
1h 48m
1
2h 24m
2
2h 7m
2
2h 20m
2
2h 30m
2
1h 42m
1
1h 59m
1
2h 17m
2
1h 36m
1
4h 2m
4
2h 23m

In [17]:
test = '2h 36m'
hours_min = test.split('h')
hours_min

['2', ' 36m']